In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

import zipfile
import os

In [2]:
class targetset(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.95 and logs.get('val_accuracy') > 0.90):
            print("\nreached")
            self.model.stop_training = True

In [3]:
import zipfile

local_zip = '/content/drive/MyDrive/capstone_needs/DATASET/clean_datav4.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/clean_datav4')
zip_ref.close()

In [5]:
train_dir = '/content/clean_datav4'
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   shear_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical',
        subset='training')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 18849 images belonging to 4 classes.
Found 4711 images belonging to 4 classes.


In [ ]:
model = Sequential([
    # Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3), kernel_regularizer=regularizers.l2(0.0001)),
    # MaxPooling2D(2, 2),
    # Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    # MaxPooling2D(2, 2),
    # Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    # MaxPooling2D(2, 2),
    # Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    # MaxPooling2D(2, 2),
    # Flatten(),
    # Dropout(0.5),
    # Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    # Dense(train_generator.num_classes, activation='softmax')
    tf.keras.Input((224,224,3)),
    tf.keras.applications.MobileNetV2(weights="imagenet", include_top=False),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(4, activation='softmax')
])

callback = targetset()
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

batch_size = 32
history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples // batch_size,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples // batch_size,
      verbose=2, callbacks=callback)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import load_img, img_to_array
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

uploaded = files.upload()

for filename in uploaded.keys():
    path = '/content/' + filename
    img = load_img(path, target_size=(224, 224))
    x = img_to_array(img) / 255
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=32)

    print(classes[0])  # printing the probabilities

    # Find the index of the highest probability
    max_index = np.argmax(classes[0])

    # Define your class names in order
    class_names = ["b3", "organic", "recyclable", "residue"]

    # Check if the highest probability is above a threshold, say 0.5
    if classes[0][max_index] > 0.5:
        print(f"{filename} is most likely '{class_names[max_index]}' with a probability of {classes[0][max_index]}")
    else:
        print(f"{filename} classification is uncertain")

In [ ]:
model.save('model4c.h5')

In [ ]:
from tensorflow.keras.models import load_model, model_from_json
model_h5 = load_model('/content/model4c.h5')
model_json = model_h5.to_json()
with open('model4c.json', 'w') as json_file:
    json_file.write(model_json)